### Tweet Cleaner

In [1]:
#import sys
#!{sys.executable} -m pip install langdetect

In [2]:
from tqdm import tqdm_notebook
import pandas as pd
import os
import pickle
import re
import numpy as np
from langdetect import detect
from langdetect import DetectorFactory
DetectorFactory.seed = 0
os.getcwd()

'C:\\Users\\Gamer\\Desktop\\hist_tweets\\GetOldTweets-python'

In [23]:
import multiprocessing as mp
cores=mp.cpu_count()
print("Number of processors: ", cores)

Number of processors:  8


In [4]:
with open('tweety_060919.pkl', 'rb') as filehandle:  
    # read the data as binary data stream
    df_tweet = pickle.load(filehandle)

In [5]:
df_tweet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2563803 entries, 0 to 2563802
Data columns (total 6 columns):
id          object
date        datetime64[ns]
text        object
author      object
favorite    int64
url         object
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 136.9+ MB


In [6]:
# check for NaN's
df_tweet.isnull().sum().sum()

0

In [7]:
# check for duplicated rows
duplicateRowsDF = df_tweet[df_tweet.duplicated()]
print("Duplicate Rows based on a all column are:", duplicateRowsDF, sep='\n')

Duplicate Rows based on a all column are:
Empty DataFrame
Columns: [id, date, text, author, favorite, url]
Index: []


In [8]:
#sample dataframe
df_tweet.sample(10)

,id,date,text,author,favorite,url
890420,328542186620727297,2013-04-28 09:12:15,@ kristiano drink the Euro-blues away,joonian,0,https://twitter.com/joonian/status/32854218662...
698751,1093167484343967744,2019-02-06 07:20:12,Prospective issuance says: looks bad for Pink ...,zooko,17,https://twitter.com/zooko/status/1093167484343...
681984,795037840774144002,2016-11-05 15:59:26,"… Perhaps contrary to intuition, being obvious...",zooko,7,https://twitter.com/zooko/status/7950378407741...
1825145,66553899515002880,2011-05-06 10:24:20,"I'm at Live Fit (969 Valencia St, 21st, San Fr...",nlw,0,https://twitter.com/nlw/status/66553899515002880
554540,897516354743545856,2017-08-15 10:52:28,I hope Bitcoin will evolve into Snack Man when...,TuurDemeester,139,https://twitter.com/TuurDemeester/status/89751...
793273,907271288816893955,2017-09-11 08:55:06,"e.g. they don't have a clear business model, o...",VladZamfir,1,https://twitter.com/VladZamfir/status/90727128...
1477393,828255692943421440,2017-02-05 06:55:20,"Funny, but relative potential for getting hack...",jonmatonis,2,https://twitter.com/jonmatonis/status/82825569...
1552478,935221109380743168,2017-11-27 10:57:42,Super high HODL ratio distorts things,KyleSamani,0,https://twitter.com/KyleSamani/status/93522110...
1005314,1073312972162777088,2018-12-13 12:25:28,Maybe! But that conversation is quite a while ...,danheld,1,https://twitter.com/danheld/status/10733129721...
1268757,456047087873163264,2014-04-15 05:31:13,@ kentindell Ha.,TheStalwart,0,https://twitter.com/TheStalwart/status/4560470...


### Determine Text Language

In [9]:
# check a non-English sample
detect(df_tweet['text'][730228])

'es'

In [10]:
# check an English sample
detect(df_tweet['text'][2466811])

'en'

In [11]:
df_test = df_tweet.sample(160).copy()
df_test.index = np.arange(len(df_test))
df_test.tail()

,id,date,text,author,favorite,url
155,824673564599189507,2017-01-26 09:41:14,"I like it a lot, further improvements are on t...",TuurDemeester,1,https://twitter.com/TuurDemeester/status/82467...
156,1097395703100379138,2019-02-17 23:21:38,pic.twitter.com/0ssj7vvONc,CarpeNoctom,11,https://twitter.com/CarpeNoctom/status/1097395...
157,466200508689309696,2014-05-13 05:57:17,"@ TylerManiac One shop makes brand new frames,...",jgarzik,0,https://twitter.com/jgarzik/status/46620050868...
158,1037748428573429760,2018-09-06 10:04:40,CIV is Castle Island Ventures (http://castleis...,nic__carter,9,https://twitter.com/nic__carter/status/1037748...
159,915878610866769920,2017-10-05 02:57:31,"But the cool thing is, if we’re all wrong and ...",udiWertheimer,0,https://twitter.com/udiWertheimer/status/91587...


In [16]:
# define language find function
def langfind(text):
    try:
        lang = detect(text[1])
    except:
        lang = 'unknown'
    return lang

In [17]:
# import exported function for multiprocessing
import langdet

In [21]:
# create the multiprocessing pool
pool = mp.Pool(cores)

# process the tweet text
with mp.Pool(cores) as pool:
    result = pool.map(langdet.langfind, df_test['text'].iteritems())

# close down the pool
pool.close()

In [24]:
print(len(result))

160


In [25]:
# split giant dataframe into 29 with 88407 rows each
df_tw = df_tweet.copy()
df_split = np.array_split(df_tw, 29, axis=0)
len(df_split)

29

In [26]:
df_split[0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88407 entries, 0 to 88406
Data columns (total 6 columns):
id          88407 non-null object
date        88407 non-null datetime64[ns]
text        88407 non-null object
author      88407 non-null object
favorite    88407 non-null int64
url         88407 non-null object
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 4.7+ MB


In [27]:
# detect languages and partition into github friendly file sizes
for i in tqdm_notebook(range(29)):
    
    df_proc = df_split[i]
    
    # create the multiprocessing pool
    pool = mp.Pool(cores)

    # process the tweet text
    with mp.Pool(cores) as pool:
        result = pool.map(langdet.langfind, df_proc['text'].iteritems())

    # close down the pool
    pool.close()
    
    # write result to column
    df_proc['lang'] = result
    
    # output to pickle
    with open('twit_lang{}.pkl'.format(i), 'wb') as filehandle:  
        # store the data
        pickle.dump(df_proc, filehandle)     

In [59]:
df_proc.tail(10)

,id,date,text,author,favorite,url,lang
2563793,1134837107996803072,2019-06-01 08:00:25,I remember being poor at times as a kid so tha...,garrytan,4,https://twitter.com/garrytan/status/1134837107...,en
2563794,1134834740937105408,2019-06-01 07:51:01,I fund teams when they are two or three people...,garrytan,3,https://twitter.com/garrytan/status/1134834740...,en
2563795,1134833351783661569,2019-06-01 07:45:29,Bitcoin in that respect is a real directional ...,garrytan,3,https://twitter.com/garrytan/status/1134833351...,en
2563796,1134829187003326464,2019-06-01 07:28:56,It’s a matter of degrees. We are really far of...,garrytan,0,https://twitter.com/garrytan/status/1134829187...,en
2563797,1134685080880680961,2019-05-31 21:56:19,You don’t mistakenly become great. Before you ...,garrytan,155,https://twitter.com/garrytan/status/1134685080...,en
2563798,1134657131561476097,2019-05-31 20:05:15,pic.twitter.com/Hoqk6XMkgn,garrytan,9,https://twitter.com/garrytan/status/1134657131...,en
2563799,1134647983180410883,2019-05-31 19:28:54,It’s not totally clear to me what Microsoft wo...,garrytan,11,https://twitter.com/garrytan/status/1134647983...,en
2563800,1134647724425375744,2019-05-31 19:27:52,Yeah Microsoft under Gates was rightly feared....,garrytan,0,https://twitter.com/garrytan/status/1134647724...,en
2563801,1134633988985778177,2019-05-31 18:33:18,Aw man,garrytan,5,https://twitter.com/garrytan/status/1134633988...,tl
2563802,1134620998047739904,2019-05-31 17:41:40,Microsoft underperformed for more than 17 year...,garrytan,175,https://twitter.com/garrytan/status/1134620998...,en


### Clean the Text

In [93]:
def cleaner(text):
    # take out space after @ symbol
    clear = re.sub(r'@\s+',r'@', text)
    # take out usernames
    clear = re.sub(r'@([A-Za-z0-9_]+)','', clear)
    # take out space after # symbol
    clear = re.sub(r'#\s+',r'#', clear)
    # take out space after $ symbol
    clear = re.sub(r'$\s+',r'$', clear)
    # take out url
    clear = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','', clear) 
    # take out pics
    clear = re.sub(r'pic.twitter?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','', clear)
    # take out the extra white spaces
    clear = re.sub(r' +', ' ', clear.strip())
    return clear

In [94]:
# test the cleaner
cleaner('@ threadreaderapp unraveled  message https://www.google.co.uk/amp/s/ ...pic.twitter.com/Hoqk6XMkgn # CourageAward')

'unraveled message #CourageAward'

In [98]:
# clean the dataframes, filter out the non-English entries
for i in tqdm_notebook(range(29)):
    
    with open('twit_lang{}.pkl'.format(i), 'rb') as filehandle:  
    # read the data as binary data stream
        df_proc = pickle.load(filehandle)
        
    # subset by English
    df_eng = df_proc[df_proc['lang']=='en'].copy()
    # clean each text block
    length = []
    cleantext = []
    for j,nrows in tqdm_notebook(df_eng.iterrows()):
        clean = cleaner(nrows['text'])
        length.append(len(clean))
        cleantext.append(clean)
    
    df_eng['length'] = length
    df_eng['cleantext'] = cleantext
    
    # output to pickle
    with open('tweet_clean{}.pkl'.format(i), 'wb') as filehandle:  
        # store the data
        pickle.dump(df_eng, filehandle)

In [99]:
df_eng['cleantext'].sample(20)

2494449      The possible prosecution of WikiLeaks | Antiwar
2510572    Snowden, Greenwald and Wikileaks are winning |...
2553930    Let's not get greedy with these bubblicious se...
2556035    Eponym lets any brand sell their own line of e...
2531188    Far worse than that. Assange is a political pr...
2484733    2/The smaller the margin you have, the more co...
2508913      #Assange works to be healthy in embassy | AAP …
2554081    If you're a fan of 2001, you will be blown awa...
2538134    I really enjoyed this conversation "weedwizard...
2535470    This deep epiphany came to me on a $ unny $ un...
2479833    Noah: A LO is an object in an OC cache. Demo f...
2523370    False dichotomy. An informed public and free m...
2531268    No wired. It's not an open question. The tape ...
2538739    I started contributing to Bitcoin Core a few m...
2505347    ..continue to pump up the insitutional "meant ...
2508289    WikiLeaks leaks film script about WikiLeaks - ...
2497580    ANNOUNCE: Use

In [100]:
df_eng.sample(10)

,id,date,text,author,favorite,url,lang,length,cleantext
2548837,1174575258,2009-02-03 15:22:12,RT: @ SFBART: BART rider Zoe for sharing a ver...,garrytan,0,https://twitter.com/garrytan/status/1174575258,en,95,RT: : BART rider Zoe for sharing a very cool p...
2549711,1675368691,2009-05-01 19:22:59,Things you learn from customer support emails:...,garrytan,1,https://twitter.com/garrytan/status/1675368691,en,106,Things you learn from customer support emails:...
2478834,1116038330305732608,2019-04-10 11:00:47,"Mondragon, all the Second Industrial Divide st...",leashless,1,https://twitter.com/leashless/status/111603833...,en,133,"Mondragon, all the Second Industrial Divide st..."
2532374,633348250418970625,2015-08-17 11:42:44,Here's why fibonacci retracements are my favou...,CryptoHustle,4,https://twitter.com/CryptoHustle/status/633348...,en,102,Here's why fibonacci retracements are my favou...
2504577,254085492063227904,2012-10-04 22:07:56,"Sweden's intelligence service, SÄPO, 'accident...",wikileaks,12,https://twitter.com/wikileaks/status/254085492...,en,85,"Sweden's intelligence service, SÄPO, 'accident..."
2551711,10400071558,2010-03-12 17:26:56,No SXSW for me. I'm at HXHW -> hack by hack we...,garrytan,0,https://twitter.com/garrytan/status/10400071558,en,95,No SXSW for me. I'm at HXHW -> hack by hack we...
2524197,710131795556175872,2016-03-16 08:53:08,Stand by for the launch of a new search system...,wikileaks,97,https://twitter.com/wikileaks/status/710131795...,en,123,Stand by for the launch of a new search system...
2536327,894607253671743489,2017-08-07 10:12:44,If you're running or planning on running a $ C...,CryptoHustle,107,https://twitter.com/CryptoHustle/status/894607...,en,139,If you're running or planning on running a $ C...
2503770,235382666751533056,2012-08-14 07:29:35,Bradley # Manning suffered unprecedented pretr...,wikileaks,8,https://twitter.com/wikileaks/status/235382666...,en,95,Bradley #Manning suffered unprecedented pretri...
2547227,1055531336331640833,2018-10-25 11:47:36,Thanks for following up. This does sound like ...,Poloniex,0,https://twitter.com/Poloniex/status/1055531336...,en,226,Thanks for following up. This does sound like ...
